In [21]:
import os
import cv2
import numpy as np

In [22]:
#browser_name = "LayerNone"
browser_names = ['LayerExtob', 'LayerSkin', 'LayerSpchd']
data_folder = r"C:\Users\PerkLab\Desktop\HerniaModelStudy\NpyData3Class"
image_name = "Webcam_Webcam"
#name_prefix = "layer_"
under_sampling_factor = 1

test_split_ratio = 0.10
shuffle_test = True
cropped_size = 256
scaled_size = 128

if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    print("Created data folder: " + data_folder)

In [23]:
# Make sure input sequence browser and video exists


for browser_name in browser_names: 
    browser_node = slicer.util.getFirstNodeByName(browser_name, className='vtkMRMLSequenceBrowserNode')
    if browser_node is None:
        logging.error("Could not find browser node: {}".format(browser_name))
        raise
    else:
        print("Found: " + browser_name)

image_node = slicer.util.getFirstNodeByName(image_name)
if image_node is None:
    logging.error("Could not find image node: {}".format(image_name))
    raise
    

Found: LayerExtob
Found: LayerSkin
Found: LayerSpchd


In [24]:
def get_train_test_from_browser_node(browser_name, tissue_class, class_num):
    browser_node = slicer.util.getFirstNodeByName(browser_name, className='vtkMRMLSequenceBrowserNode')
    n = browser_node.GetNumberOfItems()//under_sampling_factor
    #print("Number of images in browser: {}".format(n))
    
    n_test = int(n * test_split_ratio)
    all_indices = range(n)
    
    
    
    if shuffle_test:
        test_indices = np.random.choice(all_indices, n_test, replace=False) # replace must be false or error, out of range
    else:
        test_indices = range(n_test)
    
    browser_node.SelectFirstItem()
    trainCount = 0
    testCount = 0
    
    train_arrays = np.zeros((len(all_indices) - len(test_indices), scaled_size, scaled_size, 3), dtype=np.float32)
    test_arrays = np.zeros((len(test_indices), scaled_size, scaled_size, 3), dtype = np.float32)
    print(train_arrays.shape)
    print(test_arrays.shape)
    
    for i in range(n):
        image = image_node.GetImageData()
        shape = list(image.GetDimensions())
        shape.reverse()
        components = image.GetNumberOfScalarComponents()
        if components > 1:
            shape.append(components)
            shape.remove(1)
        image_array = vtk.util.numpy_support.vtk_to_numpy(image.GetPointData().GetScalars()).reshape(shape)
        try:
            if i in test_indices: 
                test_arrays[testCount] = cv2.resize(image_array[70:70+cropped_size, 150:150+cropped_size], dsize=(scaled_size, scaled_size))
                testCount+=1
            else:
                train_arrays[trainCount] = cv2.resize(image_array[70:70+cropped_size, 150:150+cropped_size], dsize=(scaled_size, scaled_size))
                trainCount+=1
        except IndexError:
            print("n = {}".format(n))
            print("i = {}".format(i))
            print("n_test = {}".format(n_test))
            print("len(all_indices) = {}".format(len(all_indices)))
            print("len(test_indices) = {}".format(len(test_indices)))
            print("trainCount = {}".format(trainCount))
            print("testCount = {}".format(testCount))
            print("n = {}".format(n))
            return 0,0
            
        for x in range(under_sampling_factor):
            browser_node.SelectNextItem()
        slicer.app.processEvents()
    
    train_y = np.full(train_arrays.shape[0], class_num, dtype = np.float32)
    test_y = np.full(test_arrays.shape[0], class_num, dtype = np.float32)
    
    print("Exported {} images of class {} for train. ".format(trainCount, tissue_class))
    print("Exported {} images of class {} for test. ".format(testCount, tissue_class))
    return train_arrays, test_arrays, train_y, test_y
    

In [25]:
#x_train, x_test, y_train, y_test = get_train_test_from_browser_node(browser_names[0], "Extob", 0)
#print(x_train.shape)
#print(y_train.shape)
#print(x_test.shape)
#print(y_test.shape)
#None.shape

In [26]:
#print(x_test.dtype)

In [27]:
for count in range(len(browser_names)):
    if count==0:
        x_train, x_test, y_train, y_test = get_train_test_from_browser_node(browser_names[count], browser_names[count], count)
    else:
        x_train_new, x_test_new, y_train_new, y_test_new = get_train_test_from_browser_node(browser_names[count], browser_names[count], count)
        x_train = np.concatenate((x_train, x_train_new))
        x_test = np.concatenate((x_test, x_test_new))
        y_train = np.concatenate((y_train, y_train_new))
        y_test = np.concatenate((y_test, y_test_new))
    print(count)
print(x_train.shape)
print(y_train.shape)

(2158, 128, 128, 3)
(239, 128, 128, 3)
Exported 2158 images of class LayerExtob for train. 
Exported 239 images of class LayerExtob for test. 
0
(2723, 128, 128, 3)
(302, 128, 128, 3)
Exported 2723 images of class LayerSkin for train. 
Exported 302 images of class LayerSkin for test. 
1
(1969, 128, 128, 3)
(218, 128, 128, 3)
Exported 1969 images of class LayerSpchd for train. 
Exported 218 images of class LayerSpchd for test. 
2
(6850, 128, 128, 3)
(6850,)


In [28]:
np.save(os.path.join(data_folder, "x_train_first_128.npy"), x_train)
np.save(os.path.join(data_folder, "x_test_first_128.npy"), x_test)
np.save(os.path.join(data_folder, "y_train_first_128.npy"), y_train)
np.save(os.path.join(data_folder, "y_test_first_128.npy"), y_test)

# def returnMany():
#     return np.array([1,2,3]), np.array([4,5,6])
# z = np.array([9,8,7,])
# np.concatenate(z, returnMany())

In [29]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(6850, 128, 128, 3)
(759, 128, 128, 3)
(6850,)
(759,)


In [30]:
# Prepare test image indices








    #image_name = name_prefix + "%06d" % i + ".npy"
    
        #image_fullname = os.path.join(output_training_folder, image_name)
    # write to file
    #image_brg = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    # print(image_brg)
    #np.save(image_fullname, image_array)

#print("Exported {} image in {}".format(testCount, output_testing_folder))
#print("Exported {} image in {}".format((trainCount), output_training_folder))